<a href="https://colab.research.google.com/github/garys-io/crispy-robot/blob/main/Calgary_communities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
# !pip install requests
import requests
from pprint import pprint as p
from bs4 import BeautifulSoup
import pathlib

In [131]:
dataDir = '/content/data'

In [132]:
## Gather data
calgaryCommuniitesUrl = 'https://www.calgary.ca/csps/cns/research-and-strategy/community-profiles/community-profiles.html'

r = requests.get(calgaryCommuniitesUrl)
# p(r.text)

In [133]:
# parse
soup = BeautifulSoup(r.text, 'html.parser')
aTags = soup.find_all('a')
print(f"Found {len(aTags)} anchor tags")

communityPageUrls = [t.attrs['href'] for t in aTags
                     if 'href' in t.attrs and t.attrs['href'].startswith('https://www.calgary.ca/csps/cns/social-research-policy-and-resources/community-profiles/')]
print(f"Found {len(communityPageUrls)} community page URLs")

# p(aTagsForCommunities)

Found 378 anchor tags
Found 197 community page URLs


In [ ]:
# Gather data for individual community by downloading their profile pdf
for communityPageUrl in communityPageUrls:
  p(communityPageUrl)
  r = requests.get(communityPageUrl)
  soup = BeautifulSoup(r.text, 'html.parser')
  pdfUrls = [t.attrs['href'] for t in soup.find_all('a')
             if t.attrs['href'] and t.attrs['href'].startswith('/content/')]
  print(f"Found {len(pdfUrls)} pdf urls")

  for pdfUrl in pdfUrls:
    url = f"https://www.calgary.ca{pdfUrl}"
    filename = pdfUrl.split('/')[-1]
    with open(f'{dataDir}/{filename}', 'wb') as f:
      r = requests.get(url)
      f.write(r.content)
    break # only first pdf is important

'https://www.calgary.ca/csps/cns/social-research-policy-and-resources/community-profiles/abbeydale-profile.html'
Found 2 pdf urls
'https://www.calgary.ca/csps/cns/social-research-policy-and-resources/community-profiles/acadia-profile.html'
Found 2 pdf urls
'https://www.calgary.ca/csps/cns/social-research-policy-and-resources/community-profiles/albert-park-radisson-heights-profile.html'
Found 2 pdf urls
'https://www.calgary.ca/csps/cns/social-research-policy-and-resources/community-profiles/altadore-profile.html'
Found 2 pdf urls
'https://www.calgary.ca/csps/cns/social-research-policy-and-resources/community-profiles/applewood-park-profile.html'
Found 2 pdf urls
'https://www.calgary.ca/csps/cns/social-research-policy-and-resources/community-profiles/arbour-lake-profile.html'
Found 2 pdf urls
'https://www.calgary.ca/csps/cns/social-research-policy-and-resources/community-profiles/aspen-woods-profile.html'
Found 2 pdf urls
'https://www.calgary.ca/csps/cns/social-research-policy-and-resour

In [ ]:
import os
os.dir('/content')